<a href="https://colab.research.google.com/github/maevedunne99/python/blob/main/Export_from_sheet_sample_data_quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
import pandas as pd
from google.auth import default
from google.colab import auth

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet = gc.open_by_key('1ecJn1g7JdQlSxhmq7J7nPlRpxUNEIB6iUFKgAVtF-gs')
worksheet = spreadsheet.get_worksheet(0)
df = pd.DataFrame(worksheet.get())

# Code for making the first row as header. Remove if not needed.
df.columns = df.iloc[0]
df = df.drop(0)
df.head()

,Analyst,Review_Result,Review_Level_1,Review_Level_2,Delivery_Date
1,Bob Sanders,Approved,34,33,2026-01-01
2,Bob Sanders,Revised,0,1,2026-01-01
3,Lisa Turner,Approved,30,30,2026-01-01
4,Lisa Turner,Revised,1,4,2026-01-01
5,Mark Evans,Approved,35,36,2026-01-01


# Pivot & Reshape Data

In [ ]:
# Dataset Structure

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 1 to 16
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Analyst         16 non-null     object
 1   Review_Result   16 non-null     object
 2   Review_Level_1  16 non-null     object
 3   Review_Level_2  16 non-null     object
 4   Delivery_Date   16 non-null     object
dtypes: object(5)
memory usage: 772.0+ bytes


In [ ]:
# Ensure numeric columns are numeric
df["Review_Level_1"] = pd.to_numeric(df["Review_Level_1"], errors="coerce").fillna(0)
df["Review_Level_2"] = pd.to_numeric(df["Review_Level_2"], errors="coerce").fillna(0)

# Pivot table
pivot = df.pivot_table(
    index="Analyst",
    columns="Review_Result",
    values=["Review_Level_1", "Review_Level_2"],
    aggfunc="sum",
    fill_value=0
)

# Flatten multi-index columns
pivot.columns = [
    f"{level}_{result.replace(' ', '_')}"
    for level, result in pivot.columns
]

pivot = pivot.reset_index()

pivot.head()

,Analyst,Review_Level_1_Approved,Review_Level_1_Revised,Review_Level_2_Approved,Review_Level_2_Revised
0,Anna Collins,32,3,35,0
1,Bob Sanders,34,0,33,1
2,David Brooks,31,0,34,0
3,Emily Carter,29,6,28,9
4,James Cooper,11,1,15,0


In [ ]:
# Calculate totals
pivot["Total_Tasks"] = pivot["Review_Level_2_Approved"] + pivot["Review_Level_2_Revised"]
pivot["L1_Total"] = pivot["Review_Level_1_Approved"] + pivot["Review_Level_1_Revised"]

# Coverage %
pivot["L1_Coverage_%"] = np.where(
    pivot["Total_Tasks"] == 0,
    0,
    pivot["L1_Total"] / pivot["Total_Tasks"]
)

# Final Quality %
pivot["Final_Quality_%"] = np.where(
    pivot["Total_Tasks"] == 0,
    0,
    pivot["Review_Level_2_Approved"] / pivot["Total_Tasks"]
)

# Delta
pivot["Revised_Delta"] = pivot["Review_Level_2_Revised"] - pivot["Review_Level_1_Revised"]

pivot.head()

,Analyst,Review_Level_1_Approved,Review_Level_1_Revised,Review_Level_2_Approved,Review_Level_2_Revised,Total_Tasks,L1_Total,L1_Coverage_%,Final_Quality_%,Revised_Delta
0,Anna Collins,32,3,35,0,35,35,1.000000,1.000000,-3
1,Bob Sanders,34,0,33,1,34,34,1.000000,0.970588,1
2,David Brooks,31,0,34,0,34,31,0.911765,1.000000,0
3,Emily Carter,29,6,28,9,37,35,0.945946,0.756757,3
4,James Cooper,11,1,15,0,15,12,0.800000,1.000000,-1


In [ ]:
analyst_summary = pivot[
    [
        "Analyst",
        "Review_Level_1_Approved",
        "Review_Level_1_Revised",
        "Review_Level_2_Approved",
        "Review_Level_2_Revised",
        "Total_Tasks",
        "L1_Total",
        "L1_Coverage_%",
        "Final_Quality_%",
        "Revised_Delta"
    ]
]

# Sort by Total Tasks descending
analyst_summary = analyst_summary.sort_values(
    by="Total_Tasks", ascending=False
).reset_index(drop=True)

# Before exporting, validate logic:
assert all(
    analyst_summary["Total_Tasks"] ==
    analyst_summary["Review_Level_2_Approved"] + analyst_summary["Review_Level_2_Revised"]
)

# Compute Team Totals

In [ ]:
# Aggregate totals from analyst_summary
total_tasks = analyst_summary["Total_Tasks"].sum()
total_level_1_reviews = analyst_summary["L1_Total"].sum()
total_level_2_approved = analyst_summary["Review_Level_2_Approved"].sum()
total_level_2_revised = analyst_summary["Review_Level_2_Revised"].sum()
worsening_count = (analyst_summary["Revised_Delta"] > 0).sum()

# Safe division helper
def safe_div(numerator, denominator):
    return numerator / denominator if denominator != 0 else 0

# Build dictionary
team_metrics = {
    "Total Tasks": total_tasks,
    "Review_Level_1 Coverage %": safe_div(total_level_1_reviews, total_tasks),
    "Review_Level_2_Approved %": safe_div(total_level_2_approved, total_tasks),
    "Review_Level_2_Revised": safe_div(total_level_2_revised, total_tasks),
    "Quality Regression Signals": worsening_count
}

# Convert to DataFrame
team_summary = pd.DataFrame(
    list(team_metrics.items()),
    columns=["Metric", "Value"]
)

team_summary.head()

,Metric,Value
0,Total Tasks,260.000000
1,Review_Level_1 Coverage %,0.953846
2,Review_Level_2_Approved %,0.930769
3,Review_Level_2_Revised,0.069231
4,Quality Regression Signals,4.000000


# Write both summary tables back to Google Sheets

In [ ]:
def write_df_to_sheet(spreadsheet, sheet_name, df):
    """
    Writes a pandas DataFrame to a Google Sheet tab.
    If the sheet does not exist, it creates it.
    If it exists, it clears and overwrites.
    """

    # Get spreadsheet ID
    spreadsheet_id = spreadsheet.id

    # Get existing sheets
    metadata = spreadsheet.fetch_sheet_metadata()
    existing_sheets = [s["properties"]["title"] for s in metadata["sheets"]]

    requests = []

    # If sheet does not exist, create it
    if sheet_name not in existing_sheets:
        requests.append({
            "addSheet": {
                "properties": {
                    "title": sheet_name
                }
            }
        })
    else:
        # Clear existing sheet
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": next(
                        s["properties"]["sheetId"]
                        for s in metadata["sheets"]
                        if s["properties"]["title"] == sheet_name
                    )
                },
                "fields": "*"
            }
        })

    # Execute structural updates first
    if requests:
        spreadsheet.batch_update({"requests": requests})

    # Prepare values (header + rows)
    values = [df.columns.tolist()] + df.values.tolist()

    # Write values
    spreadsheet.values_update(
        f"{sheet_name}!A1",
        params={"valueInputOption": "RAW"},
        body={"values": values}
    )

    print(f"✅ {sheet_name} written successfully.")

In [ ]:
write_df_to_sheet(spreadsheet, "Analyst_Summary", analyst_summary)
write_df_to_sheet(spreadsheet, "Team_Summary", team_summary)

✅ Analyst_Summary written successfully.
✅ Team_Summary written successfully.


# Create dashboard tab in Google Sheets

In [ ]:
def create_dashboard_tab(spreadsheet, dashboard_name="Dashboard"):
    """
    Deletes the Dashboard sheet if it exists,
    recreates it fresh,
    and returns the new dashboard sheetId.
    """

    metadata = spreadsheet.fetch_sheet_metadata()
    sheets = metadata["sheets"]

    existing_dashboard_sheet_id = None

    # Check if dashboard exists
    for s in sheets:
        if s["properties"]["title"] == dashboard_name:
            existing_dashboard_sheet_id = s["properties"]["sheetId"]
            break

    requests = []

    # If exists → delete it
    if existing_dashboard_sheet_id is not None:
        requests.append({
            "deleteSheet": {
                "sheetId": existing_dashboard_sheet_id
            }
        })
        print(f"Deleted existing '{dashboard_name}' sheet.")

    # Add fresh dashboard sheet
    requests.append({
        "addSheet": {
            "properties": {
                "title": dashboard_name,
                "gridProperties": {
                    "rowCount": 100,
                    "columnCount": 20
                }
            }
        }
    })

    # Execute batch update for deletion/creation
    spreadsheet.batch_update({"requests": requests})

    # Fetch metadata again to get new sheetId
    updated_metadata = spreadsheet.fetch_sheet_metadata()

    dashboard_sheet_id = None
    for s in updated_metadata["sheets"]:
        if s["properties"]["title"] == dashboard_name:
            dashboard_sheet_id = s["properties"]["sheetId"]
            break

    if dashboard_sheet_id is None:
        raise ValueError(f"Could not find sheetId for '{dashboard_name}' after creation.")

    # Now that dashboard_sheet_id is known, hide gridlines
    spreadsheet.batch_update({
        "requests": [
            {
                "updateSheetProperties": {
                    "properties": {
                        "sheetId": dashboard_sheet_id,
                        "gridProperties": {
                            "hideGridlines": True
                        }
                    },
                    "fields": "gridProperties.hideGridlines"
                }
            }
        ]
    })

    print(f"✅ '{dashboard_name}' created successfully.")
    print(f"Dashboard sheetId: {dashboard_sheet_id}")

    return dashboard_sheet_id

In [ ]:
dashboard_sheet_id = create_dashboard_tab(spreadsheet)

✅ 'Dashboard' created successfully.
Dashboard sheetId: 481475269


 # Write KPIs to the dashboard

In [38]:
total_tasks = team_metrics["Total Tasks"]
level_1_reviews_coverage = team_metrics["Review_Level_1 Coverage %"]
level_2_approved = team_metrics["Review_Level_2_Approved %"]
worsening_count = team_metrics["Quality Regression Signals"]

level_1_reviews_coverage_pct = round(level_1_reviews_coverage  * 100, 1)
level_2_approved_pct = round(level_2_approved * 100, 1)

# KPIs for dashboard
def write_kpis_to_dashboard(spreadsheet, dashboard_name,
                            total_tasks,
                            level_1_reviews_coverage_pct,
                            level_2_approved_pct,
                            worsening_count):

    # Ensure JSON-serializable types
    total_tasks = int(total_tasks)
    level_1_reviews_coverage_pct = float(level_1_reviews_coverage_pct)
    level_2_approved_pct = float(level_2_approved_pct)
    worsening_count = int(worsening_count)

    values = [
        ["Total Tasks", "", "", "Level 1 Reviews Coverage %", "", "", "Level 2 Approved %", "", "", "⚠ Quality Regression Signals"],
        [total_tasks, "", "", f"{level_1_reviews_coverage_pct}%", "", "", f"{level_2_approved_pct}%", "", "", worsening_count]
    ]

    spreadsheet.values_update(
        f"{dashboard_name}!B1",
        params={"valueInputOption": "RAW"},
        body={"values": values}
    )

    print("✅ KPI tiles written to Dashboard.")

In [39]:
# Call the function
write_kpis_to_dashboard(
    spreadsheet,
    "Dashboard",
    total_tasks,
    level_1_reviews_coverage_pct,
    level_2_approved_pct,
    worsening_count
)

✅ KPI tiles written to Dashboard.


In [40]:
# Formatting:
def format_kpi_section(spreadsheet, dashboard_sheet_id):
    requests = [
        # Format KPI labels (Row 1)
        {
            "repeatCell": {
                "range": {
                    "sheetId": dashboard_sheet_id,
                    "startRowIndex": 0,
                    "endRowIndex": 1,
                    "startColumnIndex": 0,
                    "endColumnIndex": 12
                },
                "cell": {
                    "userEnteredFormat": {
                        "backgroundColor": {
                            "red": 0.9,
                            "green": 0.9,
                            "blue": 0.9
                        },
                        "horizontalAlignment": "CENTER",
                        "textFormat": {
                            "bold": True,
                            "fontSize": 16
                        }
                    }
                },
                "fields": "userEnteredFormat(backgroundColor,horizontalAlignment,textFormat)"
            }
        },

        # Format KPI values (Row 2)
        {
            "repeatCell": {
                "range": {
                    "sheetId": dashboard_sheet_id,
                    "startRowIndex": 1,
                    "endRowIndex": 2,
                    "startColumnIndex": 0,
                    "endColumnIndex": 12
                },
                "cell": {
                    "userEnteredFormat": {
                      "backgroundColor": {
                        "red": 0.98,
                        "green": 0.94,
                        "blue": 0.94
                        },
                        "horizontalAlignment": "CENTER",
                        "textFormat": {
                            "bold": True,
                            "fontSize": 20
                        }
                    }
                },
                "fields": "userEnteredFormat(backgroundColor,horizontalAlignment,textFormat)"
            }
        },

        # Increase row height for KPI value row
        {
            "updateDimensionProperties": {
                "range": {
                    "sheetId": dashboard_sheet_id,
                    "dimension": "ROWS",
                    "startIndex": 1,
                    "endIndex": 2
                },
                "properties": {
                    "pixelSize": 60
                },
                "fields": "pixelSize"
            }
        }
    ]

    spreadsheet.batch_update({"requests": requests})

    print("✅ KPI formatting applied.")

# Call the function
format_kpi_section(spreadsheet, dashboard_sheet_id)

✅ KPI formatting applied.


# Add Charts to the Dashboard tab

In [41]:
# Automate retrieval of sheet ID
def get_sheet_id(spreadsheet, sheet_name):
    """
    Returns the sheetId for a given sheet name.
    Raises an error if not found.
    """
    metadata = spreadsheet.fetch_sheet_metadata()

    for sheet in metadata["sheets"]:
        if sheet["properties"]["title"] == sheet_name:
            return sheet["properties"]["sheetId"]

    raise ValueError(f"Sheet '{sheet_name}' not found.")

analyst_sheet_id = get_sheet_id(spreadsheet, "Analyst_Summary")

In [44]:
# === Create chart 1 ===

def add_workload_distribution_chart(spreadsheet,
                                    dashboard_sheet_id,
                                    analyst_sheet_id,
                                    analyst_df):
    """
    Adds Workload Distribution chart (Analyst vs Total_Tasks)
    to the Dashboard sheet at position A5.
    """

    start_row = 1  # skip header
    end_row = len(analyst_df) + 1

    request = {
        "addChart": {
            "chart": {
                "spec": {
                    "title": "Workload Distribution",
                    "titleTextPosition": {
                        "horizontalAlignment": "CENTER"
                    },
                    "basicChart": {
                        "chartType": "COLUMN",
                        "legendPosition": "NO_LEGEND", # Changed from "NONE" to "NO_LEGEND"
                        "axis": [
                            {"position": "BOTTOM_AXIS", "title": "Analyst"},
                            {"position": "LEFT_AXIS", "title": "Total Tasks"}
                        ],
                        "domains": [{
                            "domain": {
                                "sourceRange": {
                                    "sources": [{
                                        "sheetId": analyst_sheet_id,
                                        "startRowIndex": start_row,
                                        "endRowIndex": end_row,
                                        "startColumnIndex": 0,   # SLS
                                        "endColumnIndex": 1
                                    }]
                                }
                            }
                        }],
                        "series": [{
                            "series": {
                                "sourceRange": {
                                    "sources": [{
                                        "sheetId": analyst_sheet_id,
                                        "startRowIndex": start_row,
                                        "endRowIndex": end_row,
                                        "startColumnIndex": 5,   # Total_Tasks
                                        "endColumnIndex": 6
                                    }]
                                }
                            }
                        }]
                    }
                },
                "position": {
                    "overlayPosition": {
                        "anchorCell": {
                            "sheetId": dashboard_sheet_id,
                            "rowIndex": 4,
                            "columnIndex": 0
                        },
                        "offsetXPixels": 0,
                        "offsetYPixels": 0,
                        "widthPixels": 650,
                        "heightPixels": 350
                    }
                }
            }
        }
    }

    spreadsheet.batch_update({"requests": [request]})

    print("✅ Workload Distribution chart added.")

# Call function
add_workload_distribution_chart(
    spreadsheet=spreadsheet,
    dashboard_sheet_id=dashboard_sheet_id,
    analyst_sheet_id=analyst_sheet_id,
    analyst_df=analyst_summary)

✅ Workload Distribution chart added.


In [47]:
# === Create chart 2 ===

def add_tier3_quality_chart(
    spreadsheet,
    dashboard_sheet_id,
    analyst_sheet_id,
    analyst_row_count
):
    """
    Creates a 100% stacked column chart showing
    Review_Level_2_Approved (green) vs Review_Level_2_Revised (red) per Analyst
    """

    requests = [
        {
            "addChart": {
                "chart": {
                    "spec": {
                        "title": "Level 2 Quality (%)",
                        "titleTextPosition": {
                        "horizontalAlignment": "CENTER"
                        },
                        "basicChart": {
                            "chartType": "COLUMN",
                            "legendPosition": "TOP_LEGEND", # Changed to "NO_LEGEND"
                            "stackedType": "PERCENT_STACKED",
                            "axis": [
                                {
                                    "position": "BOTTOM_AXIS",
                                    "title": "Analyst"
                                },
                                {
                                    "position": "LEFT_AXIS",
                                    "title": "Percent"
                                }
                            ],
                            "domains": [
                                {
                                    "domain": {
                                        "sourceRange": {
                                            "sources": [
                                                {
                                                    "sheetId": analyst_sheet_id,
                                                    "startRowIndex": 0,     # include header
                                                    "endRowIndex": analyst_row_count,
                                                    "startColumnIndex": 0,
                                                    "endColumnIndex": 1
                                                }
                                            ]
                                        }
                                    }
                                }
                            ],
                            "series": [
                                {
                                    "series": {
                                        "sourceRange": {
                                            "sources": [
                                                {
                                                    "sheetId": analyst_sheet_id,
                                                    "startRowIndex": 0,   # include header for legend
                                                    "endRowIndex": analyst_row_count,
                                                    "startColumnIndex": 3,
                                                    "endColumnIndex": 4
                                                }
                                            ]
                                        }
                                    },
                                    "color": {
                                        "red": 0.2,
                                        "green": 0.7,
                                        "blue": 0.2
                                    }
                                },
                                {
                                    "series": {
                                        "sourceRange": {
                                            "sources": [
                                                {
                                                    "sheetId": analyst_sheet_id,
                                                    "startRowIndex": 0,
                                                    "endRowIndex": analyst_row_count,
                                                    "startColumnIndex": 4,
                                                    "endColumnIndex": 5
                                                }
                                            ]
                                        }
                                    },
                                    "color": {
                                        "red": 0.8,
                                        "green": 0.2,
                                        "blue": 0.2
                                    }
                                }
                            ],
                            "headerCount": 1
                        }
                    },
                    "position": {
                        "overlayPosition": {
                            "anchorCell": {
                                "sheetId": dashboard_sheet_id,
                                "rowIndex": 4,
                                "columnIndex": 7
                            },
                            "widthPixels": 650,
                            "heightPixels": 350
                        }
                    }
                }
            }
        }
    ]

    spreadsheet.batch_update({"requests": [requests]})

    print("✅ Tier 3 % chart added.")

# Assign a value to analyst_row_count before calling the chart function

analyst_row_count = len(analyst_summary) + 1

# Call function
add_tier3_quality_chart(
    spreadsheet=spreadsheet,
    dashboard_sheet_id=dashboard_sheet_id,
    analyst_sheet_id=analyst_sheet_id,
    analyst_row_count=analyst_row_count
)

✅ Tier 3 % chart added.


In [51]:
# === Create chart 3 ===

def create_tier2_coverage_chart(spreadsheet, dashboard_sheet_id, analyst_sheet_id):

    requests = [
        {
            "addChart": {
                "chart": {
                    "spec": {
                        "title": "Level 1 Reviews Coverage %",
                        "titleTextPosition": {
                            "horizontalAlignment": "CENTER"
                        },
                        "basicChart": {
                            "chartType": "BAR",
                            "legendPosition": "NO_LEGEND", # Changed from "NONE" to "NO_LEGEND"
                            "axis": [
                                {
                                    "position": "BOTTOM_AXIS",
                                    "title": "Coverage %"
                                },
                                {
                                    "position": "LEFT_AXIS",
                                    "title": "Analyst"
                                }
                            ],
                            "domains": [
                                {
                                    "domain": {
                                        "sourceRange": {
                                            "sources": [
                                                {
                                                    "sheetId": analyst_sheet_id,
                                                    "startRowIndex": 1,
                                                    "startColumnIndex": 0,
                                                    "endColumnIndex": 1
                                                }
                                            ]
                                        }
                                    }
                                }
                            ],
                            "series": [
                                {
                                    "series": {
                                        "sourceRange": {
                                            "sources": [
                                                {
                                                    "sheetId": analyst_sheet_id,
                                                    "startRowIndex": 1,
                                                    "startColumnIndex": 7,
                                                    "endColumnIndex": 8
                                                }
                                            ]
                                        }
                                    }
                                }
                            ]
                        }
                    },
                    "position": {
                        "overlayPosition": {
                            "anchorCell": {
                                "sheetId": dashboard_sheet_id,
                                "rowIndex": 23,
                                "columnIndex": 0
                            },
                            "offsetXPixels": 0,
                            "offsetYPixels": 0,
                            "widthPixels": 650,
                            "heightPixels": 350
                        }
                    }
                }
            }
        }
    ]

    spreadsheet.batch_update({"requests": [requests]})

    print("✅ Chart created.")

def format_t2_coverage(spreadsheet, analyst_sheet_id):

    requests = [

        # Format as percentage
        {
            "repeatCell": {
                "range": {
                    "sheetId": analyst_sheet_id,
                    "startRowIndex": 1,
                    "startColumnIndex": 7,
                    "endColumnIndex": 8
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "PERCENT",
                            "pattern": "0.0%"
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        },

        # Sort by coverage descending
        {
            "sortRange": {
                "range": {
                    "sheetId": analyst_sheet_id,
                    "startRowIndex": 1,
                    "startColumnIndex": 0,
                    "endColumnIndex": 8
                },
                "sortSpecs": [
                    {
                        "dimensionIndex": 7,
                        "sortOrder": "DESCENDING"
                    }
                ]
            }
        }

    ]

    spreadsheet.batch_update({"requests": [requests]})

    print("T2 coverage formatted and sorted.")

# Call functions
create_tier2_coverage_chart(spreadsheet, dashboard_sheet_id, analyst_sheet_id)

format_t2_coverage(spreadsheet, analyst_sheet_id)

✅ Chart created.
T2 coverage formatted and sorted.


In [53]:
# === Create chart 4 ===

def create_t2_t3_edit_distribution_chart(spreadsheet, dashboard_sheet_id, analyst_sheet_id):

    requests = [
        {
            "addChart": {
                "chart": {
                    "spec": {
                        "title": "Quality Regression (L1 vs L2 Revised)",
                        "titleTextPosition": {
                            "horizontalAlignment": "CENTER"
                        },
                        "basicChart": {
                            "chartType": "COLUMN",
                            "legendPosition": "TOP_LEGEND",
                            "headerCount": 1,
                            "axis": [
                                {
                                    "position": "BOTTOM_AXIS",
                                    "title": "Analyst"
                                },
                                {
                                    "position": "LEFT_AXIS",
                                    "title": "Revised Count"
                                }
                            ],
                            "domains": [
                                {
                                    "domain": {
                                        "sourceRange": {
                                            "sources": [
                                                {
                                                    "sheetId": analyst_sheet_id,
                                                    "startRowIndex": 0,
                                                    "startColumnIndex": 0,
                                                    "endColumnIndex": 1
                                                }
                                            ]
                                        }
                                    }
                                }
                            ],
                            "series": [
                                {
                                    "series": {
                                        "sourceRange": {
                                            "sources": [
                                                {
                                                    "sheetId": analyst_sheet_id,
                                                    "startRowIndex": 0,
                                                    "startColumnIndex": 2,
                                                    "endColumnIndex": 3
                                                }
                                            ]
                                        }
                                    },
                                    "targetAxis": "LEFT_AXIS",
                                    "color": {
                                        "red": 0.26,
                                        "green": 0.52,
                                        "blue": 0.96
                                    }
                                },
                                {
                                    "series": {
                                        "sourceRange": {
                                            "sources": [
                                                {
                                                    "sheetId": analyst_sheet_id,
                                                    "startRowIndex": 0,
                                                    "startColumnIndex": 4,
                                                    "endColumnIndex": 5
                                                }
                                            ]
                                        }
                                    },
                                    "targetAxis": "LEFT_AXIS",
                                    "color": {
                                        "red": 0.85,
                                        "green": 0.22,
                                        "blue": 0.22
                                    }
                                }
                            ]
                        }
                    },
                    "position": {
                        "overlayPosition": {
                            "anchorCell": {
                                "sheetId": dashboard_sheet_id,
                                "rowIndex": 23,
                                "columnIndex": 7
                            },
                            "widthPixels": 650,
                            "heightPixels": 350
                        }
                    }
                }
            }
        }
    ]

    spreadsheet.batch_update({"requests": [requests]})

    print("✅ T2 vs T3 Work Edited chart created.")

# Call function
create_t2_t3_edit_distribution_chart(spreadsheet, dashboard_sheet_id, analyst_sheet_id)

✅ T2 vs T3 Work Edited chart created.


# Add timestamp

In [54]:
# === Create timestamp ===

from datetime import datetime
from zoneinfo import ZoneInfo

def add_dashboard_timestamp(spreadsheet, dashboard_name, dashboard_sheet_id):

    cst_time = datetime.now(ZoneInfo("America/Mexico_City"))
    timestamp = cst_time.strftime("%d %b %Y | %H:%M CST")

    # Write the timestamp
    spreadsheet.values_update(
        f"{dashboard_name}!N1",
        params={"valueInputOption": "RAW"},
        body={"values": [[f"Last Updated: {timestamp}"]]}
    )

    # 2Format the cell
    requests = [
        {
            "repeatCell": {
                "range": {
                    "sheetId": dashboard_sheet_id,
                    "startRowIndex": 0,
                    "endRowIndex": 1,
                    "startColumnIndex": 13,  # Column N
                    "endColumnIndex": 14
                },
                "cell": {
                    "userEnteredFormat": {
                        "horizontalAlignment": "CENTER",
                        "textFormat": {
                            "fontSize": 9,
                            "italic": True,
                            "foregroundColor": {
                                "red": 0.5,
                                "green": 0.5,
                                "blue": 0.5
                            }
                        }
                    }
                },
                "fields": "userEnteredFormat(horizontalAlignment,textFormat)"
            }
        }
    ]

    spreadsheet.batch_update({"requests": [requests]})

    print("✅ Dashboard timestamp added and formatted.")

# Call function
add_dashboard_timestamp(spreadsheet, "Dashboard", dashboard_sheet_id)

✅ Dashboard timestamp added and formatted.
